# Охват первой дозой вакцины, содержащей коревой компонент (MCV1), среди детей в возрасте 1 года (%)

In [9]:
import pandas as pd

from helpers import (
    get_data_with_full_locations_and_years, 
    get_omissions,
)

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/measles-containing-vaccine-first-dose-(mcv1)-immunization-coverage-among-1-year-olds-(-)

In [10]:
# Загрузим таблицу
data = pd.read_csv('../data/who_measles_immunization.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WHS8_110,Measles-containing-vaccine first-dose (MCV1) i...,numeric,EUR,Europe,Country,MNE,Montenegro,Year,2020,...,NaN,NaN,NaN,NaN,NaN,24,NaN,NaN,EN,2024-07-11T21:00:00.000Z
1,WHS8_110,Measles-containing-vaccine first-dose (MCV1) i...,numeric,AFR,Africa,Country,CAF,Central African Republic,Year,2020,...,NaN,NaN,NaN,NaN,NaN,41,NaN,NaN,EN,2024-07-11T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - охват первой дозой вакцины, содержащей коревой компонент (MCV1), среди детей в возрасте 1 года (%)

Оставим только их.

In [11]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

data.rename(columns={'FactValueNumeric': 'MeaslesImmunization'}, inplace=True)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,MeaslesImmunization
0,EUR,MNE,2020,24
1,AFR,CAF,2020,41
2,AFR,AGO,2020,44
3,WPR,PNG,2020,44
4,AFR,TCD,2020,45


В данной таблице для некоторых стран данные заполнены не за все года.\
Добавим строки с отсутствующими годами, чтобы явно увидеть пропуски.

In [12]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на количество пропусков
data.isna().sum()

ParentLocationCode      0
SpatialDimValueCode     0
Period                  0
MeaslesImmunization    19
dtype: int64

In [13]:
# Посмотрим на пропущенные данные
omissions_data = get_omissions(data)

omissions_data.sort_values(by=['SpatialDimValueCode', 'Period'])

,ParentLocationCode,SpatialDimValueCode,Period,MeaslesImmunization
3900,EUR,MNE,2000,NaN
3705,EUR,MNE,2001,NaN
3510,EUR,MNE,2002,NaN
3315,EUR,MNE,2003,NaN
3120,EUR,MNE,2004,NaN
2925,EUR,MNE,2005,NaN
3927,AFR,SSD,2000,NaN
3732,AFR,SSD,2001,NaN
3537,AFR,SSD,2002,NaN
3342,AFR,SSD,2003,NaN


Заполним их средним по региону.

In [14]:
data['MeaslesImmunization'] = \
    data.groupby(['ParentLocationCode', 'Period'])['MeaslesImmunization'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [15]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [16]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_measles_immunization_prepared.csv', index=False)